In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/Installment/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(3200, 2)


,label,split_text
0,2,这样
1,0,好
2,2,我 很 忙 对不起
3,0,好 的 我 有 足够 的 时间 来 支付 今晚 。
4,2,不 不 不
5,0,那么 你 不能 回去
6,0,行 等 下 就 还 。
7,1,延期 三个 月 我 也 还 不了
8,1,你 能 减少 它 吗 ？
9,0,在 开始 时 我 开始 接受 它 。


In [3]:
data.split_text.iloc[42]

'估计 不是'

In [4]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<3200x7327 sparse matrix of type '<class 'numpy.float64'>'
	with 24991 stored elements in Compressed Sparse Row format>

# SVC

In [5]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.9853125


In [6]:
print(clf.predict(phrase))

[2 0 2 ..., 1 0 0]


# logistic

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.96


In [9]:
print(log_r.predict(phrase))

[2 0 2 ..., 1 0 0]


# LightGBM

In [10]:
import lightgbm as lgb

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [12]:
onelabels

array([2, 0, 2, ..., 1, 0, 0])

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [14]:
lables

array([[1, 1, 1]])

In [15]:
data.label.values

array([2, 0, 2, ..., 1, 0, 0])

In [16]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.201563
[10]	valid_0's multi_error: 0.175625
[15]	valid_0's multi_error: 0.16125
[20]	valid_0's multi_error: 0.14875
[25]	valid_0's multi_error: 0.142187
[30]	valid_0's multi_error: 0.138125
[35]	valid_0's multi_error: 0.132188
[40]	valid_0's multi_error: 0.127812
[45]	valid_0's multi_error: 0.126562
[50]	valid_0's multi_error: 0.123125
[55]	valid_0's multi_error: 0.119375
[60]	valid_0's multi_error: 0.115625
[65]	valid_0's multi_error: 0.113437
[70]	valid_0's multi_error: 0.112187
[75]	valid_0's multi_error: 0.109063
[80]	valid_0's multi_error: 0.108125
[85]	valid_0's multi_error: 0.107188
[90]	valid_0's multi_error: 0.107188
[95]	valid_0's multi_error: 0.10625
[100]	valid_0's multi_error: 0.104375
[105]	valid_0's multi_error: 0.104688
[110]	valid_0's multi_error: 0.10375
[115]	valid_0's multi_error: 0.101562
[120]	valid_0's multi_error: 0.10125
[125]	valid_0's multi_error: 0.100937
[130]	valid_

In [17]:
print(lgbm_model.predict(phrase))

[[ 0.60564801  0.02884043  0.40655115]
 [ 0.96171411  0.00565314  0.04658061]
 [ 0.00772318  0.00333949  0.98398717]
 ..., 
 [ 0.00582912  0.91962595  0.07665217]
 [ 0.63220403  0.38849943  0.00579255]
 [ 0.98675579  0.02646844  0.01586792]]


In [18]:
phrase

<3200x7327 sparse matrix of type '<class 'numpy.float64'>'
	with 24991 stored elements in Compressed Sparse Row format>

# Save

In [19]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/Installment/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/Installment/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/Installment/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/Installment/Lgbm.pickle", "wb"))

# Test

In [20]:
test = phrase_vectorizer.transform(['我 真的 没有 钱 呀'])
# test = phrase_vectorizer.transform(['我 在 洗澡'])


In [21]:
clf.predict_proba(test) # linear svc

array([[ 0.0086741,  0.9632555,  0.0280704]])

In [22]:
log_r.predict_proba(test) # logistic

array([[ 0.06787473,  0.80033033,  0.13179494]])

In [23]:
lgbm_model.predict(test) # light gbm

array([[  2.84910981e-03,   9.98920197e-01,   8.55871215e-04]])

# Labeling

In [24]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.998920196558


# Save the whole thing

In [25]:
# import jieba
# import numpy as np

# class Installment:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred)<threshold:
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [26]:
%load_ext autoreload
%autoreload 2

In [27]:
from Installment_py import Installment
ifk = Installment(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/Installment/Installment.pickle", "wb"))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.450 seconds.
Prefix dict has been built succesfully.
